In [1]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from scipy import io as sio
from tensorflow.python.framework import ops
from dfs2 import DeepFeatureSelectionNew
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import normalize

ourdataB = sio.loadmat("/Volumes/TONY/Regeneron/Data/OriginalData/newDataB_2labels.mat")
inputX = ourdataB['X']
inputX = normalize(inputX, axis=0)
inputY = ourdataB['Y'][0,:]
columnNames = ourdataB['columnNames']

# iris = datasets.load_iris()
# inputX = iris.data[:,[1,2,3]]
# inputY = iris.target

# digits = datasets.load_digits()
# inputX = digits.data  
# inputY = digits.target

# inputX, inputY = make_classification(n_samples=1000, n_features=7500, n_informative=3000, n_redundant=0, n_repeated=0, n_classes=2)

X_train, X_test, y_train, y_test = train_test_split(inputX, inputY, test_size=0.2, random_state=42)

In [2]:
indexes = sio.loadmat("xgboost_result")['importance_rank']

In [3]:
X_train500, X_test500 = X_train[:, indexes.tolist()[0][:500]], X_test[:, indexes.tolist()[0][:500]]
    
X_train10, X_test10 = X_train[:, indexes.tolist()[0][:10]], X_test[:, indexes.tolist()[0][:10]]

In [ ]:
len(indexes.tolist()[0][:500])

In [5]:
ops.reset_default_graph()

dfsMLP = DeepFeatureSelectionNew(X_train500, X_test500, y_train, y_test, n_input=0, hidden_dims=[300], learning_rate=0.01, \
                                 lambda1=0.001, lambda2=0.1, alpha1=0.01, alpha2=0.05, activation='sigmoid', \
                                 weight_init='mlp',epochs=100000, optimizer='Adam', print_step=5)

dfsMLP.train(batch_size=1000)

epoch 0: global loss = 14.4005060196
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 5: global loss = 1.21139788628
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 10: global loss = 0.974212765694
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 15: global loss = 0.951200664043
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 20: global loss = 0.934549212456
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 25: global loss = 0.91210103035
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 30: global loss = 0.895365476608
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 35: global loss = 0.877996623516
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 40: global loss = 0.864667713642
('Train accuracy:', 0.57131159)
('Test accuracy:', 0.56621432)
epoch 45: global loss = 0.84937864542
('Train accuracy:', 0.57131159)
('Test accuracy:'

KeyboardInterrupt: 

In [7]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

batch_size = 2000
nb_classes = 2
nb_epoch = 500

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()
model.add(Dense(300, input_shape=(500,)))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train500, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test500, Y_test))
score = model.evaluate(X_test500, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_3 (Dense)                  (None, 300)           150300      dense_input_2[0][0]              
____________________________________________________________________________________________________
activation_3 (Activation)        (None, 300)           0           dense_3[0][0]                    
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 2)             602         activation_3[0][0]               
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 2)             0           dense_4[0][0]                    
Total params: 150902
______________________________________________________________________

KeyboardInterrupt: 

In [ ]:
Y_train

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

temp = abs(dfsMLP.selected_ws[0])

plt.plot(temp)

In [ ]:
temp = dfsMLP.selected_ws

temp1, temp2 = temp[0], temp[1]

In [ ]:
np.where(abs(temp1)<0.01)

In [ ]:
np.where(abs(temp2)<0.01)

In [ ]:
temp = dfsMLP.selected_ws

temp3, temp4 = temp[0], temp[1]

print(np.where(abs(temp3)<0.01))
print(np.where(abs(temp4)<0.01))

In [ ]:
temp = dfsMLP.selected_ws

temp5, temp6 = temp[0], temp[1]

print(np.where(abs(temp5)<0.01))
print(np.where(abs(temp6)<0.01))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

# rf = RandomForestClassifier(criterion="entropy", n_estimators = 300, max_depth = 100)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=400, learning_rate=0.05).fit(X_train, y_train)
y_pred = gbm.predict(X_test)

print(accuracy_score(y_test, y_pred))

# gbm10 = xgb.XGBClassifier(max_depth=3, n_estimators=400, learning_rate=0.05).fit(X_train10, y_train)
# y_pred10 = gbm.predict(X_test10)

# print(accuracy_score(y_test, y_pred))

In [ ]:
indexes = np.argsort(gbm.feature_importances_)[::-1]

# top100Features = np.array([columnNames[0][i] for i in indexes[0:200]])
# top100Features = np.array([columnNames[i] for i in indexes[0:100]])
# top100Features = top100Features.reshape(100,)

In [ ]:
X_train5, X_test5 = X_train[:, indexes[:5]], X_test[:, indexes[:5]]

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1")

clf.fit(X_train500, y_train)

y_pred500lr = clf.predict(X_test500)

print(accuracy_score(y_test, y_pred500lr))